In [1]:
import pandas as pd

In [61]:
f = open("imrmc_reader_study.imrmc", "r")

# Convert iMRMC to Pandas
df = pd.DataFrame(columns=['ReaderID','CaseID','ModalityID','Rating'])
vals = False
for l in f.readlines():
    if l.startswith("-1"):
        vals=True
    if vals:
        l = l.strip()
        reader_id, case_id, modality_id, rating = l.split(",")
        df = df.append({
            "ReaderID": int(reader_id),
            "CaseID": int(case_id),
            "ModalityID": modality_id,
            "Rating": float(rating)
        }, ignore_index=True)

df

,ReaderID,CaseID,ModalityID,Rating
0,-1,1,truth,0.0
1,-1,2,truth,0.0
2,-1,3,truth,0.0
3,-1,4,truth,0.0
4,-1,5,truth,0.0
...,...,...,...,...
1395,6,196,1,0.0
1396,6,197,1,0.0
1397,6,198,1,0.0
1398,6,199,1,0.0


In [62]:
# Convert Dataframe to RJafroc format

# Worksheet "TRUTH"
truth_df = df[df.ReaderID==-1][['CaseID', 'Rating']]
truth_df = truth_df.rename(columns={"Rating": "LesionID"})
truth_df['LesionID'] = truth_df['LesionID'].apply(int)
truth_df['Weight'] = 0
truth_df

,CaseID,LesionID,Weight
0,1,0,0
1,2,0,0
2,3,0,0
3,4,0,0
4,5,0,0
...,...,...,...
195,196,0,0
196,197,0,0
197,198,1,0
198,199,0,0


In [63]:
# Worksheet "TP" (only diseased cases)
tp_df = df[(
    (df.CaseID.isin(truth_df[truth_df.LesionID==1].CaseID))
    & (df.ReaderID!=-1)
)]
tp_df = tp_df.rename(columns={"Rating": "LL_Rating"})
tp_df['LesionID'] = 1
tp_df['ModalityID'] = tp_df.ModalityID.apply(int)
tp_df = tp_df[['ReaderID', 'ModalityID', 'CaseID', 'LesionID', 'LL_Rating']]

tp_df

,ReaderID,ModalityID,CaseID,LesionID,LL_Rating
209,1,1,10,1,95.836805
210,1,1,11,1,69.304576
218,1,1,19,1,93.665525
224,1,1,25,1,83.273933
225,1,1,26,1,76.151334
...,...,...,...,...,...
1388,6,1,189,1,70.000000
1389,6,1,190,1,85.000000
1391,6,1,192,1,90.000000
1393,6,1,194,1,85.000000


In [64]:
# Worksheet "FP" (only negative cases)
fp_df = df[(
    (df.CaseID.isin(truth_df[truth_df.LesionID==0].CaseID))
    & (df.ReaderID!=-1)
)]
fp_df = fp_df.rename(columns={"Rating": "NL_Rating"})
fp_df['ModalityID'] = fp_df.ModalityID.apply(int)
fp_df = fp_df[['ReaderID', 'ModalityID', 'CaseID', 'NL_Rating']]

fp_df

,ReaderID,ModalityID,CaseID,NL_Rating
200,1,1,1,1.321749
201,1,1,2,24.300648
202,1,1,3,2.709108
203,1,1,4,23.732458
204,1,1,5,25.939885
...,...,...,...,...
1394,6,1,195,0.000000
1395,6,1,196,0.000000
1396,6,1,197,0.000000
1398,6,1,199,0.000000


In [65]:
#writer = pd.ExcelWriter('dataset09_converted.xlsx')
writer = pd.ExcelWriter('reader_study_converted.xlsx')
tp_df.to_excel(writer, sheet_name='TP', index=False)
fp_df.to_excel(writer, sheet_name='FP', index=False)
truth_df.to_excel(writer, sheet_name='TRUTH', index=False)
writer.save()